# Inteligencia Artificial
## Algoritmos Genéticos

### Integrantes
- Aldana S. Cardoso Rastrelli - 98.xxx
- Nicolás Continanza - 97576


## Breve Marco Histórico

Dentro del campo de la Inteligencia Artificial y la resolución automática de problemas, en la década del '60 John Holland y su equipo de la Universidad de Michigan presentan los algoritmos genéticos, inspirados en la evolución biológica y la selección natural para problemas computacionales y de investigación operativa. El objetivo original de Holland no era el diseño de algoritmos para la resolución de problemas específicos, sino el estudio formal del fenómeno de adaptación como sucede en la naturaleza y el desarrollo de técnicas y métodos para importar mecanismos de adaptación natural a sistemas de computadoras.

En su libro _Adaptation in Natural and Artificial Systems (1975)_, Holland presenta un algoritmo genético como un método para pasar de una población de "cromosomas" (un conjunto de bits) a una nueva población evolucionada, mediante un proceso de selección natural junto con un conjunto de operadores inspirados en la genética (entrecruzamiento, mutación e inversión). Cada cromosoma consiste de "genes" (bits), cada uno siendo una instancia de un "alelo" (0 ó 1). El operador de selección elige aquellos cromosomas de la población que podrán reproducirse, y en promedio los cromosomas más aptos producen más descendencia que los menos aptos. El operador de crossover (entrecruzamiento, también llamado recombinación en buena parte de la literatura) intercambia subpartes de dos cromosomas, imitando la recombinación cromosómica entre dos haploides (células con un solo juego de cromosomas). El operador de mutación cambia aleatoriamente el valor de los alelos de algunas posiciones en el cromosoma. Y el operador de inversión invierte el orden de una sección contigua del cromosoma, cambiando el orden de los genes.

Con el pasar de los años hubo una amplia interacción entre investigadores que se dedicaron a estudiar variados métodos evolutivos computacionales, corriendo cada vez más los límites entre algoritmos genéticos, estrategias evolutivas y programación evolutiva, al punto de que al día de hoy el término _algoritmo genético_ se utiliza para describir cosas bastante lejanas a la concepción original de Holland, incorporando técnicas de machine learning para encontrar buenas (y a veces incluso óptimas) soluciones a problemas combinatorios con una inmensa cantidad de soluciones posibles. 

## Un primer ejemplo

Los algoritmos genéticos utilizan la exploración aleatoria del espacio del problema, combinando procesos evolutivos como la mutación y el crossover para mejorar las conjeturas. Pero también, al no tener experiencia en el dominio del problema, intentan cosas que un humano nunca intentaría. De esta manera, una persona usando un algoritmo genético puede aprender más acerca del espacio del problema y potenciales soluciones, pudiendo así realizar mejoras al algoritmo en un círculo virtuoso.

Tomemos como ejemplo inicial el ejercicio de adivinar una contraseña. Para simplificar el problema y poner el foco en lo que nos interesa, conocemos de antemano la longitud de la contraseña. Para ayudar a introducir algunos conceptos iniciales, después de cada intento tendremos la posibilidad de saber cuántas letras estaban en la posición correcta. Por ejemplo, si la contraseña es "Hello World!" y se intenta con la cadena "World!Hello?" se obtendrá el número 2, ya que solo la cuarta letra de cada palabra está en la posición correcta. 

Pseudocódigo:
```
_letters = [a..zA..Z !]
target = "Hello World!"
guess = get 12 random letters from _letters
while guess != target:
  index = get random value from [0..length of target]
  guess[index] = get 1 random value from _letters
```

### Genes

Para comenzar, los algoritmos genéticos necesitan un set de genes para construir las posibles soluciones. Para este ejemplo inicial, el set de genes serán las letras del abecedario. También es necesario un target (la contraseña a adivinar):



In [8]:
geneSet = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!."
target = "Hello World!"

A continuación, el algoritmo deberá tener una manera de generar una cadena aleatoria a partir del set de genes.

In [9]:
import random

def generate_parent(length): 
    genes = []
    while len(genes) < length:
        sampleSize = min(length - len(genes), len(geneSet)) 
        genes.extend(random.sample(geneSet, sampleSize))
    return ''.join(genes)


La función `random.sample()` toma una cantidad `sampleSize` de valores de la entrada sin reposición. Por lo tanto no habrá valores repetidos en el parent generado, salvo que `geneSet` contenga repetidos o `length` sea mayor que `len(geneSet)`. Esta implementación puede generar una larga cadena de caracteres a partir de un pequeño conjunto de genes, y usa tantos genes únicos como sea posible.

### Función de Aptitud (Fitness Function)

El valor de fitness que provee el algoritmo genético es el único feedback que el sistema obtiene para guiarse hacia una solución. En este ejemplo, el valor de fitness es la cantidad total de letras en un intento que coinciden con la letra del target en la misma posición.

In [10]:
def get_fitness(guess):
    return sum(1 for expected, actual in zip(target, guess) 
               if expected == actual)


### Mutación

A continuación, el sistema requiere de una forma de producir un nuevo intento mutando el actual. La siguiente función convierte la cadena `parent` en un array, y luego reemplaza una letra del array con una elegida aleatoriamente de `geneSet`, recombinando el resultado nuevamente en una cadena de caracteres.

In [11]:
def mutate(parent):
    index = random.randrange(0, len(parent)) 
    childGenes = list(parent)
    newGene, alternate = random.sample(geneSet, 2) 
    childGenes[index] = alternate \
        if newGene == childGenes[index] \
        else newGene
    return ''.join(childGenes)


Se usa un reemplazo alternativo en caso de que `newGene` (seleccionado aleatoriamente) sea igual al que se supone que reemplazará, para prevenir un número significativo de intentos irrelevantes.

### Mostrando resultados

Queremos monitorear el paso a paso del algoritmo para tener una noción de su avance y sacar conclusiones. Una representación visual de la secuencia genética suele ser crítico para identificar qué funciona y qué no, para que el algoritmo pueda ser mejorado.

En nuestro caso, mostraremos también el valor de fitness y cuánto tiempo transcurrió.

In [12]:
import datetime

def display(guess):
    timeDiff = datetime.datetime.now() - startTime
    fitness = get_fitness(guess)
    print("{0}\t{1}\t{2}".format(guess, fitness, str(timeDiff)))

Pasemos a poner en práctica nuestro algoritmo y analizar los resultados obtenidos.
En esencia, el algoritmo consistirá de los siguientes pasos:
  1. Generar un intento.
  2. Obtener el valor `fitness` para ese intento
  3. Comparar el valor `fitness` obtenido con el del intento anterior
  4. Conservar el intento con mejor `fitness`
Esto se repite en un ciclo hasta que ocurra una condición de corte, que en nuestro caso es haber encontrado la contraseña correcta (`fitness = 12`).

In [14]:
random.seed()
startTime = datetime.datetime.now()
bestParent = generate_parent(len(target))
bestFitness = get_fitness(bestParent)
display(bestParent)

while True:
    child = mutate(bestParent) 
    childFitness = get_fitness(child) 
    if bestFitness >= childFitness:
        continue
    display(child)
    if childFitness >= len(bestParent):
        break
    bestFitness = childFitness
    bestParent = child

gDnKFOaUXQeS	0	0:00:00.000143
gDnKFOaUXleS	1	0:00:00.000326
gDnKFOaoXleS	2	0:00:00.000361
gDnKoOaoXleS	3	0:00:00.000728
gDnKoOWoXleS	4	0:00:00.001883
gDnKo WoXleS	5	0:00:00.002605
gDnlo WoXleS	6	0:00:00.002962
gDllo WoXleS	7	0:00:00.003770
gello WoXleS	8	0:00:00.004342
gello WoXldS	9	0:00:00.004413
gello WoXld!	10	0:00:00.004744
Hello WoXld!	11	0:00:00.005743
Hello World!	12	0:00:00.013152


Vemos que obtener la contraseña con este elemental algoritmo genético tomó poco más de 1 centésima de segundo.


## Bibliografía
- R. Poli, W. B. Langdon, and N. F. McPhee. _A field guide to genetic programming_. 2008. (With contributions by J. R. Koza). Publicado via http://lulu.com y disponible gratuitamente en http://www.gp-field-guide.org.uk